In [ ]:
pip install openai==1.2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.2.4
    Uninstalling openai-1.2.4:
      Successfully uninstalled openai-1.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
pip install --upgrade openai

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from scipy.stats import pearsonr
import openai
import re
openai.api_key = '' # mention your openai API key here

In [ ]:
import openai
print(openai.VERSION)

1.2.4


## Data Extraction

In [ ]:
import pandas as pd
data = pd.read_csv('/content/tracie_train.txt', sep='\t',names=['event_story','answer'],header=None)
data

,event_story,answer
0,event: Chad looked for his baseball cap starts...,answer: positive
1,event: Chad looked for his baseball cap starts...,answer: negative
2,event: Tom's friend says he is not gay starts...,answer: positive
3,event: Tom's friend says he is not gay starts...,answer: negative
4,event: Tom is heterosexual starts before Tom '...,answer: positive
...,...,...
1169,event: Kelly gave her name to the hostess ends...,answer: negative
1170,event: He hired a lawyer. ends after he sued t...,answer: positive
1171,event: He hired a lawyer. ends before he sued ...,answer: negative
1172,event: The son was born. ends before I throw t...,answer: positive


In [ ]:
data[['Event', 'Context']] = data['event_story'].str.split('story:', expand=True)
data['Event']=data['Event'].str.replace('event:','')
data['answer']=data['answer'].str.replace('answer:','')
data['key']='MATRESS'
data

,event_story,answer,Event,Context,key
0,event: Valley Federal Savings amp Loan Associa...,positive,Valley Federal Savings amp Loan Association t...,Valley Federal Savings amp Loan Association ...,MATRESS
1,event: Valley Federal Savings amp Loan Associa...,negative,Valley Federal Savings amp Loan Association t...,Valley Federal Savings amp Loan Association ...,MATRESS
2,event: Valley Federal Savings amp Loan Associa...,positive,Valley Federal Savings amp Loan Association t...,Valley Federal Savings amp Loan Association ...,MATRESS
3,event: Valley Federal Savings amp Loan Associa...,negative,Valley Federal Savings amp Loan Association t...,Valley Federal Savings amp Loan Association ...,MATRESS
4,event: it reported a third - quarter loss of $...,positive,it reported a third - quarter loss of $ 70.7 ...,Valley Federal Savings amp Loan Association t...,MATRESS
...,...,...,...,...,...
20821,event: Mr. Marous said Westinghouse would own ...,negative,Mr. Marous said Westinghouse would own 70 % o...,"Westinghouse Electric Corp. , capitalizing on...",MATRESS
20822,"event: The deal , which will involve an initia...",positive,"The deal , which will involve an initial $ 20...","Westinghouse Electric Corp. , capitalizing on...",MATRESS
20823,"event: The deal , which will involve an initia...",negative,"The deal , which will involve an initial $ 20...","Westinghouse Electric Corp. , capitalizing on...",MATRESS
20824,"event: The deal , which will involve an initia...",positive,"The deal , which will involve an initial $ 20...","In 1988 , the company earned $ 822.8 million ...",MATRESS


In [ ]:
data.to_csv('Tracie_Matress.csv4',header=True,index=False)

In [ ]:
from numpy import append
data.to_csv('Tracie_Matress.csv',header=False,index=False,mode='a')

#Prompt Generation

In [ ]:
final_data=pd.read_csv('/content/Tracie_Matress.csv')
print(len(final_data))
final_data.head(2)

27900


,event_story,answer,Event,Context,key
0,event: The teacher asked us to stop talking st...,positive,The teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE
1,event: The teacher asked us to stop talking st...,negative,The teacher asked us to stop talking starts b...,I was so nervous for my first day of school. ...,TRACIE


In [ ]:
# model_name='gpt-4'
model_name='gpt-3.5-turbo'
def get_completion(prompt, model=model_name):
  messages=[{"role":"user","content":prompt}]
  response=openai.ChatCompletion.create(model=model,messages=messages,temperature=0)
  return response.choices[0].message['content']

In [ ]:
final_data['GPT3']=''
final_data['GPT4']=''
# 27570

In [ ]:
prediction=[]
for index,row in final_data.iterrows():
    prompt=  "context:'{0}'.For the given context,state if the statement: event '{1}' is True or False. Give your answer in the format , Ans:<ans>".format(row.Context,row.Event)
    response = get_completion(prompt)
    print(prompt)
    print(response)
    prediction.append(re.findall("^Ans:(.*\w*)", response)[0] if len(re.findall("^Ans:(.*\w*)", response))>0 else 'None')

context:' He apparently hopes to lure support from Arabs who have spurned him so far by suggesting that Iraq will use its occupation of Kuwait as a lever to solve the Arab world 's most frustrating problem , the 23-year Israeli occupation of land claimed by Palestinians . Mr. Baker declined to use the word blockade , but said that " interdiction " of Iraqi shipments would begin " almost instantly . In a statement , the White House said it would do " whatever is necessary " to ensure compliance with the sanctions . Other Bush administration officials said that the international naval force in the area -- consisting of American , British , French , Canadian , Soviet , German and Australian ships -- may be used both to stop oil exports from leaving Iraq and Kuwait and to stop shipments of food and other goods from going in . President Bush implied as much yesterday when reporters asked whether the interdiction would apply to food . " The naval interdiction force is part of an overall Amer

In [ ]:
final_data['GPT3']=prediction
final_data.to_csv('Implicit_TE_GPT3_5.csv', index=False, header=False,mode='a')

In [ ]:
st=pd.read_csv('Implicit_TE_GPT3_5.csv',header=None,names=['event_story','answer','Event',	'Context',	'key',	'GPT3',	'GPT4'])
print(len(st))
st.GPT3.value_counts()

7766


 True     5714
 False    2052
Name: GPT3, dtype: int64

In [ ]:
st

,event_story,answer,Event,Context,key,GPT3,GPT4
9900,event: he managing current company business st...,positive,he managing current company business starts b...,But sources said he will be urging his allie...,MATRESS,True,
9901,event: he managing current company business st...,negative,he managing current company business starts a...,But sources said he will be urging his allie...,MATRESS,True,
9902,event: he managing current company business st...,positive,he managing current company business starts b...,"At a news conference , Mr. Fournier accused P...",MATRESS,False,
9903,event: he managing current company business st...,negative,he managing current company business starts a...,"At a news conference , Mr. Fournier accused P...",MATRESS,False,
9904,event: But sources said he will be urging his ...,positive,But sources said he will be urging his allies...,But sources said he will be urging his allie...,MATRESS,True,
...,...,...,...,...,...,...,...
9995,event: Navigation Mixte said it now holds just...,negative,Navigation Mixte said it now holds just under...,Cie . de Navigation Mixte Chairman Marc Fourn...,MATRESS,True,
9996,event: it now holds just under 5 % of Paribas ...,positive,it now holds just under 5 % of Paribas 's sha...,Mr. Fournier said that as Navigation Mixte ch...,MATRESS,True,
9997,event: it now holds just under 5 % of Paribas ...,negative,it now holds just under 5 % of Paribas 's sha...,Mr. Fournier said that as Navigation Mixte ch...,MATRESS,True,
9998,event: it now holds just under 5 % of Paribas ...,positive,it now holds just under 5 % of Paribas 's sha...,Cie . de Navigation Mixte Chairman Marc Fourn...,MATRESS,False,


# GPT4

In [ ]:
final_data=pd.read_csv('/content/Implicit_TE_GPT3_5.csv')
print(len(final_data))
final_data.head(2)

23266


,event_story,answer,Event,Context,key,GPT3,GPT4
0,event: The teacher asked us to stop talking st...,positive,The teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE,True,NaN
1,event: The teacher asked us to stop talking st...,negative,The teacher asked us to stop talking starts b...,I was so nervous for my first day of school. ...,TRACIE,False,NaN


In [1]:
prediction=[]
for index,row in final_data.iterrows():
    prompt=  "context:'{0}'.For the given context,state if the statement: event '{1}' is True or False. Give your answer in the format , Ans:<ans>".format(row.Context,row.Event)
    response = get_completion(prompt)
    prediction.append(re.findall("^Ans:(.*\w*)", response)[0] if len(re.findall("^Ans:(.*\w*)", response))>0 else 'None')

# code

In [ ]:
final_data['GPT4']=prediction

In [ ]:
final_data.to_csv('Implicit_TE_GPT3_5_GPT4_prompt.csv', index=False, header=False,mode='a')

In [ ]:
st=pd.read_csv('/content/Implicit_TE_GPT3_5_GPT4_prompt.csv')
print(len(st))
st.head(5)

3147


,event_story,answer,Event,Context,key,GPT3,GPT4
0,event_story,answer,Event,Context,key,GPT3,None
1,event: The teacher asked us to stop talking st...,positive,The teacher asked us to stop talking starts a...,I was so nervous for my first day of school. ...,TRACIE,True,False
2,event: The teacher asked us to stop talking st...,negative,The teacher asked us to stop talking starts b...,I was so nervous for my first day of school. ...,TRACIE,False,False
3,event: The man was holding a piece of paper st...,positive,The man was holding a piece of paper starts b...,"John didn't know he was being followed. ""When...",TRACIE,True,True
4,event: The man was holding a piece of paper st...,negative,The man was holding a piece of paper starts a...,"John didn't know he was being followed. ""When...",TRACIE,True,False
